In [ ]:
import numpy as np
data = np.load("actions_dataset.npz")
X, y = data["X"], data["y"]

better understand the data

In [ ]:
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("\nFirst 5 elements of X:\n", X[:5])
print("\nFirst 5 elements of y:\n", y[:5])
unique_classes, class_counts = np.unique(y, return_counts=True)
unique_classes_x, class_counts_x = np.unique(X, return_counts=True)
print("\nUnique classes and their counts in y:")
for class_val, count in zip(unique_classes, class_counts):
    print(f"Class {class_val}: {count}")
print("\nUnique classes and their counts in X:")
for class_val, count in zip(unique_classes_x, class_counts_x):
    print(f"Class {class_val}: {count}")

Shape of X: (12428, 5)
Shape of y: (12428, 12)

First 5 elements of X:
 [[8 2 8 2 6]
 [2 8 2 6 2]
 [8 2 6 2 8]
 [2 6 2 8 2]
 [6 2 8 2 8]]

First 5 elements of y:
 [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

Unique classes and their counts in y:
Class 0.0: 136708
Class 1.0: 12428

Unique classes and their counts in X:
Class 0: 1115
Class 1: 1305
Class 2: 21487
Class 3: 105
Class 4: 65
Class 5: 3565
Class 6: 3805
Class 7: 2755
Class 8: 24913
Class 9: 370
Class 10: 845
Class 11: 1810


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, log_loss, top_k_accuracy_score
from sklearn.metrics import top_k_accuracy_score
# ---- inputs you already have ----
# X: shape (N, 5)   integers
# y: shape (N, 12)  one-hot

rng = np.random.default_rng(42)

# 1) prep: one-hot -> labels
y_ids = y.argmax(axis=1)
num_classes = y.shape[1]

# 2) split (use only TRAIN priors to avoid leakage)
X_tr, X_te, y_tr_ids, y_te_ids, y_tr_oh, y_te_oh = train_test_split(
    X, y_ids, y, test_size=0.2, random_state=42, stratify=y_ids
)

# 3) compute class priors from TRAIN
counts = np.bincount(y_tr_ids, minlength=num_classes).astype(float)
priors = counts / counts.sum()  # weighted-random probabilities

# optional: Laplace smoothing if any class might be missing in train
# priors = (counts + 1.0) / (counts.sum() + num_classes)

# 4) RANDOM (uniform) baseline
rand_pred_ids = rng.integers(low=0, high=num_classes, size=len(y_te_ids))
# probability matrix for log_loss/top-k
rand_probs = np.full((len(y_te_ids), num_classes), 1.0 / num_classes)

# 5) WEIGHTED-RANDOM (by priors) baseline
wrand_pred_ids = rng.choice(num_classes, size=len(y_te_ids), p=priors)
# same priors for every row
wrand_probs = np.tile(priors, (len(y_te_ids), 1))

# 6) metrics
def evaluate(y_true_ids, pred_ids, prob_mat, name="baseline", k_list=(3,5)):
    out = {
        f"{name}/accuracy": accuracy_score(y_true_ids, pred_ids),
        f"{name}/macro_f1": f1_score(y_true_ids, pred_ids, average='macro'),
        f"{name}/micro_f1": f1_score(y_true_ids, pred_ids, average='micro'),
        f"{name}/log_loss": log_loss(y_true_ids, prob_mat, labels=np.arange(num_classes)),
    }
    for k in k_list:
        out[f"{name}/top{k}"] = top_k_accuracy_score(y_true_ids, prob_mat, k=k, labels=np.arange(num_classes))
    return out

rand_metrics  = evaluate(y_te_ids, rand_pred_ids,  rand_probs,  name="random")
wrand_metrics = evaluate(y_te_ids, wrand_pred_ids, wrand_probs, name="weighted_random")

print("Class priors (from train):", np.round(priors, 4))
print("\nRANDOM:", rand_metrics)
print("\nWEIGHTED-RANDOM:", wrand_metrics)

# Example: top-3 accuracy for random baseline
top3_rand = top_k_accuracy_score(y_te_ids, rand_probs, k=3, labels=np.arange(num_classes))

# Example: top-3 accuracy for weighted-random baseline
top3_wrand = top_k_accuracy_score(y_te_ids, wrand_probs, k=3, labels=np.arange(num_classes))

print(f"Random baseline Top-3 accuracy: {top3_rand:.4f}")
print(f"Weighted-random baseline Top-3 accuracy: {top3_wrand:.4f}")

Class priors (from train): [0.0179 0.021  0.3458 0.0017 0.0011 0.0573 0.0612 0.0444 0.4009 0.0059
 0.0136 0.0292]

RANDOM: {'random/accuracy': 0.07803700724054706, 'random/macro_f1': 0.047365727419264377, 'random/micro_f1': 0.07803700724054706, 'random/log_loss': 2.484906649788001, 'random/top3': np.float64(0.048672566371681415), 'random/top5': np.float64(0.49396621078037006)}

WEIGHTED-RANDOM: {'weighted_random/accuracy': 0.2847948511665326, 'weighted_random/macro_f1': 0.08118566097749759, 'weighted_random/micro_f1': 0.2847948511665326, 'weighted_random/log_loss': 1.568482310667108, 'weighted_random/top3': np.float64(0.8081255028157683), 'weighted_random/top5': np.float64(0.909895414320193)}


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, log_loss, top_k_accuracy_score

# ---- inputs you already have ----
# X: shape (N, 5)   integers
# y: shape (N, 12)  one-hot

# use random generator (no fixed seed -> different results each run)
rng = np.random.default_rng()

# 1) prep: one-hot -> labels
y_ids = y.argmax(axis=1)
num_classes = y.shape[1]

# 2) split (use only TRAIN priors to avoid leakage)
X_tr, X_te, y_tr_ids, y_te_ids, y_tr_oh, y_te_oh = train_test_split(
    X, y_ids, y, test_size=0.2, random_state=42, stratify=y_ids
)

# 3) compute class priors from TRAIN
counts = np.bincount(y_tr_ids, minlength=num_classes).astype(float)
priors = counts / counts.sum()

# --------------------------
# Baseline probability generators
# --------------------------

def make_random_probs(n_samples, num_classes, rng):
    """Uniform random probability distribution per sample."""
    return rng.dirichlet(np.ones(num_classes), size=n_samples)

def make_weighted_random_probs(n_samples, priors, rng, concentration=50):
    """Random probs centered on class priors."""
    return rng.dirichlet(priors * concentration, size=n_samples)

# --------------------------
# Evaluation helper
# --------------------------
def evaluate(y_true_ids, prob_mat, name="baseline", k_list=(1,3,5)):
    pred_ids = prob_mat.argmax(axis=1)
    out = {
        f"{name}/accuracy": accuracy_score(y_true_ids, pred_ids),
        f"{name}/macro_f1": f1_score(y_true_ids, pred_ids, average='macro'),
        f"{name}/micro_f1": f1_score(y_true_ids, pred_ids, average='micro'),
        f"{name}/log_loss": log_loss(y_true_ids, prob_mat, labels=np.arange(num_classes)),
    }
    for k in k_list:
        out[f"{name}/top{k}"] = top_k_accuracy_score(
            y_true_ids, prob_mat, k=k, labels=np.arange(num_classes)
        )
    return out

# --------------------------
# Run baselines
# --------------------------

# Uniform random
rand_probs = make_random_probs(len(y_te_ids), num_classes, rng)
rand_metrics = evaluate(y_te_ids, rand_probs, name="random")

# Weighted random
wrand_probs = make_weighted_random_probs(len(y_te_ids), priors, rng)
wrand_metrics = evaluate(y_te_ids, wrand_probs, name="weighted_random")

print("Class priors (from train):", np.round(priors, 4))
print("\nRANDOM:", rand_metrics)
print("\nWEIGHTED-RANDOM:", wrand_metrics)


Class priors (from train): [0.0179 0.021  0.3458 0.0017 0.0011 0.0573 0.0612 0.0444 0.4009 0.0059
 0.0136 0.0292]

RANDOM: {'random/accuracy': 0.07884151246983105, 'random/macro_f1': 0.05135330319296511, 'random/micro_f1': 0.07884151246983105, 'random/log_loss': 2.9888090938761827, 'random/top1': np.float64(0.07884151246983105), 'random/top3': np.float64(0.2578439259855189), 'random/top5': np.float64(0.415526950925181)}

WEIGHTED-RANDOM: {'weighted_random/accuracy': 0.3998390989541432, 'weighted_random/macro_f1': 0.07292208200507116, 'weighted_random/micro_f1': 0.3998390989541432, 'weighted_random/log_loss': 1.7087628204160876, 'weighted_random/top1': np.float64(0.3998390989541432), 'weighted_random/top3': np.float64(0.8032984714400644), 'weighted_random/top5': np.float64(0.8849557522123894)}
